In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#the dataset
df = pd.read_csv("DIS.csv", header=None)
close_prices = df[5].values

#Reshape and normalize
close_prices = close_prices.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(close_prices)

#split into train and test
train, test = train_test_split(dataset, test_size=0.5, shuffle=False)

def create_dataset(data, look_back=240):
    X, Y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data[i:(i + look_back), 0])
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 240
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#reshape for LSTM
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(25, input_shape=(1, look_back)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

#train
model.fit(trainX, trainY, epochs=1000, batch_size=240, verbose=1)


from sklearn.metrics import mean_squared_error
import math

#predict
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

#inverse transform predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY_scaled = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY_scaled = scaler.inverse_transform([testY])

#RMSE
trainScore = math.sqrt(mean_squared_error(trainY_scaled[0], trainPredict[:, 0]))
testScore = math.sqrt(mean_squared_error(testY_scaled[0], testPredict[:, 0]))

print("Train RMSE: %.2f" % trainScore)
print("Test RMSE: %.2f" % testScore)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0096
Epoch 2/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0022  
Epoch 3/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0014 
Epoch 4/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8.7565e-04 
Epoch 5/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7.0075e-04 
Epoch 6/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.9015e-04 
Epoch 7/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5.5121e-04 
Epoch 8/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.2403e-04 
Epoch 9/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.2126e-04 
Epoch 10/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.4701e-04 
Epoch 11/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.8539e-04 
Epoch 12/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.8536e-04 
Epoch 13/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.3194e-04 
Epoch 14/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.2174e-04 